In [2]:
import os
import pandas as pd
import pickle
#from Bio import SeqIO
from Bio import AlignIO   #, Align

base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts/alphafold')
divyeast_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/diverse_yeast')
y1000plus_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools/data') + os.sep


#C:\Users\heineib\Documents\GitHub\y1000plus_tools\data\y1000plus_tools_data\y1000plus\id_lookups


In [ ]:
#Goal:  Go from filename from Oliver's script to what would be a standard filename for any generated structure.  That will then be converted to Jacob's.  
# While doing so remove from both the sequence and the protein alignment proteins that have been flagged by oliver.  

#Structural alignment fasta headers
# <orig_species_name>__<og>__<y1000_id>

# Except for C.alb, S.cer and S. pombe which have
# <species_abbrev>_AF-<swissprot_id>-F1-model_V2
#
# Unless it is the reference protein in which case it has: 

#REF_<species_abbrev>_AF-<swissprot_id>-F1-model_V2

#Sequence alignment fasta headers

#<old_species_name>|<y1000_id>

In [10]:
#species_lookup - y1000 species table
y1000_species = pd.read_csv(y1000plus_dir + os.path.normpath("y1000plus_tools_data/y1000plus/y1000_species_table.csv"), index_col=0)

#fig2_orig_genome_lookup = dict(zip(y1000_species['species_names_fig2'], y1000_species['original_genome_id']))
#old_orig_genome_lookup = dict(zip(y1000_species['old_species_names'], y1000_species['original_genome_id']))
orig_old_genome_lookup = dict(zip(y1000_species['original_genome_id'], y1000_species['old_species_names']))


spec_abbrev_dict = {'Scer': 'Saccharomyces_cerevisiae', 
                    'Calb': 'Candida_albicans', 
                    'Spom': 'Schizosaccharomyces_pombe'}


#Lookup from gene_id to y1000_id

gene_id_2_y1000_id = {}

#Load S.cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/saccharomyces_cerevisiae.csv')
scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
gene_id_2_y1000_id['Scer'] = dict(zip(scer_lookup.index,scer_lookup['y1000_id']))

#Load C.alb lookup table
calb_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/candida_albicans.csv')
calb_lookup = pd.read_csv(calb_lookup_fname, index_col=0)
gene_id_2_y1000_id['Calb'] = dict(zip(calb_lookup.index,calb_lookup['y1000_id']))

#Lookup from swissprot_id to gene_id
swissprot_id_2_gene_id= {}
scer_swissprot_id_2_gene_id_df = pd.read_table(base_dir + os.sep + os.path.normpath('msas/structural/Scer_protein_names.tsv'))
swissprot_id_2_gene_id['Scer'] = dict(zip(scer_swissprot_id_2_gene_id_df['Swiss-Prot'],scer_swissprot_id_2_gene_id_df['OLN']))

calb_swissprot_id_2_gene_id = pickle.load(open(base_dir + os.sep + os.path.normpath('msas/structural/Mapping_calb.pkl'),"rb"))
swissprot_id_2_gene_id['Calb'] = dict(zip(calb_swissprot_id_2_gene_id.values(),calb_swissprot_id_2_gene_id.keys()))


#Load analysis file which identifies bad structures
struct_analysis = pickle.load(open(base_dir + os.sep + os.path.normpath('Output/data/Analysis_new_02.pkl'), 'rb'))



In [23]:
#convert_qc = {}

keep_in_filt_seq = True #flag to skip filtered seqs that don't show up in orthogroups

for fasta_fname in os.listdir(base_dir + os.sep +  os.path.normpath('msas\structural\FASTA\\') ):
#fasta_fname = 'OG1004_REF_Scer_AF-P15938-F1-model_v2.FASTA'

    convert_qc_og_ref = {}
    og = fasta_fname.split('_')[0]
    ref = fasta_fname.split('_')[2] + '_' + fasta_fname.split('_')[3] + '_' + fasta_fname.split('_')[4].split('.')[0]

    seqs_to_remove = struct_analysis[fasta_fname.split('.')[0]]['Files to be excluded']
    
    fname_struct_aln_orig = base_dir + os.sep +  os.path.normpath('msas\structural\FASTA\\' + fasta_fname)   # os.path.normpath('msas\FILES_ogs_pep_aligned\\' + og + '.mfaa.mafft')
    fname_seq_aln_orig = base_dir + os.sep + os.path.normpath('msas\FILES_ogs_pep_aligned\\' + og + '.mfaa.mafft')

    fname_struct_aln_renamed_out = base_dir + os.sep + os.path.normpath('msas\structural\\fasta_renamed\\' + og + '__' + ref + '.struct.fasta')
    fname_struct_aln_filt_out = base_dir + os.sep + os.path.normpath('msas\structural\\fasta_renamed_filt\\' + og + '__' + ref + '.struct_filt.fasta')
    fname_seq_aln_filt_out =  base_dir + os.sep + os.path.normpath('msas\structural\\fasta_renamed_seq_filt\\' + og + '__' + ref + '.seq_filt.fasta')

    #Read in Sequence File, extract all names make dict of seq_alignment sequences
    seq_align = AlignIO.read(fname_seq_aln_orig, 'fasta')
    seq_align_names = [record.id for record in seq_align]
    seq_align_seqs = {record.id: record.seq for record in seq_align}
    
    #Read in dictionary of ref name map to sequence name
    struct_align = AlignIO.read(fname_struct_aln_orig, 'fasta')
    struct_names_converted = []
    with open(fname_struct_aln_renamed_out, 'w') as f_out: 
        with open(fname_struct_aln_filt_out , 'w') as f_out_filt:
            with open(fname_seq_aln_filt_out, 'w') as f_out_seq_filt: 
                for record in struct_align: 
                    skip = False    #Right now only skipping Spom sequences since those are not in sequence alignment
                    name_orig = record.id    

                    #Check if name in ref sequence (S.cer, C.alb, and S. pom) + convert

                    if name_orig.split('_')[0] in set(['Scer', 'REF', 'Calb', 'Spom']):
                        if name_orig.split('_')[0] == 'REF': 
                            spec_abbrev = name_orig.split('_')[1]
                            swissprot_id = name_orig.split('_')[2].split('-')[1]
                        else: 
                            spec_abbrev = name_orig.split('_')[0]
                            swissprot_id = name_orig.split('_')[1].split('-')[1]

                        if spec_abbrev == 'Spom':
                            #S pombe not currently included in sequence alignments
                            skip = True
                        else: 
                            spec_old = spec_abbrev_dict[spec_abbrev]

                            y1000_id = gene_id_2_y1000_id[spec_abbrev][swissprot_id_2_gene_id[spec_abbrev][swissprot_id]]


                    else: 
                        #Convert name            
                        (spec_orig, og, y1000_id) = name_orig.split('__')
                        spec_old = orig_old_genome_lookup[spec_orig]

                    if not(skip):

                        seq_aln_id = spec_old + '|' + y1000_id 

                        if not(seq_aln_id in seq_align_names): 
                            keep_in_filt_seq = False
                            print('in OG: ' + og + ' , REF: ' + ref + ', ' + name_orig + ' converts to ' + seq_aln_id + ' and is not in the sequence ids')

                        #Print out fasta without filter
                        struct_names_converted.append(seq_aln_id)
                        f_out.write('>' + seq_aln_id + '\n')
                        f_out.write(str(record.seq) + '\n')

                        #Filter out seqs with bad alignments
                                                
                        if not(name_orig in seqs_to_remove): 
                            f_out_filt.write('>' + seq_aln_id + '\n')
                            f_out_filt.write(str(record.seq) + '\n')
                            if keep_in_filt_seq: 
                                f_out_seq_filt.write('>' + seq_aln_id + '\n')
                                f_out_seq_filt.write(str(seq_align_seqs[seq_aln_id]) + '\n')
                            
                        keep_in_filt_seq = True
                        



    #Double check lists of names

    #seq_not_struct = list(set(seq_align_names)-set(struct_names_converted))
    #convert_qc_og_ref['seq_not_struct'] = seq_not_struct

    #convert_qc[og_ref] = convert_qc_og_ref


in OG: OG1054 , REF: Scer_AF-P16861-F1-model_v2, REF_Scer_AF-P16861-F1-model_v2 converts to Saccharomyces_cerevisiae|110_2957 and is not in the sequence ids
in OG: OG1054 , REF: Scer_AF-P16862-F1-model_v2, REF_Scer_AF-P16861-F1-model_v2 converts to Saccharomyces_cerevisiae|110_2957 and is not in the sequence ids
in OG: OG1254 , REF: Scer_AF-P40395-F1-model_v2, Calb_AF-A0A1D8PCB4-F1-model_v2 converts to Candida_albicans|12_4978 and is not in the sequence ids
in OG: OG1254 , REF: Scer_AF-Q01519-F1-model_v2, Calb_AF-A0A1D8PCB4-F1-model_v2 converts to Candida_albicans|12_4978 and is not in the sequence ids
in OG: OG1355 , REF: Scer_AF-P10614-F1-model_v2, REF_Scer_AF-P10614-F1-model_v2 converts to Saccharomyces_cerevisiae|110_3481 and is not in the sequence ids
in OG: OG1468 , REF: Scer_AF-P17898-F1-model_v2, REF_Scer_AF-P17898-F1-model_v2 converts to Saccharomyces_cerevisiae|110_204 and is not in the sequence ids
in OG: OG1468 , REF: Scer_AF-P22140-F1-model_v2, REF_Scer_AF-P17898-F1-model_

In [18]:
import json
with open(base_dir + os.sep + 'og_metadata.json', 'r') as f:
    og_metadata = json.load(f) 

In [21]:
og_metadata['OG1054']

{'N_genes': 50,
 'pct_spec': 1.0,
 'og_genes': ['0_3372',
  '0_7632',
  '0_842',
  '104_1425',
  '104_4147',
  '106_3026',
  '106_4589',
  '110_2957',
  '110_4171',
  '12_1254',
  '12_5593',
  '134_532',
  '135_1844',
  '135_604',
  '136_4557',
  '136_4679',
  '136_5128',
  '139_1297',
  '139_1946',
  '13_3097',
  '144_4509',
  '170_2397',
  '170_5931',
  '247_3196',
  '247_4481',
  '30_2127',
  '30_4309',
  '342_2471',
  '342_3054',
  '35_3053',
  '35_5019',
  '36_2507',
  '36_5240',
  '40_3514',
  '40_367',
  '43_4452',
  '43_767',
  '49_1933',
  '49_2285',
  '49_4805',
  '50_1049',
  '50_1134',
  '51_494',
  '51_705',
  '52_1129',
  '52_3574',
  '64_1246',
  '64_645',
  '66_3500',
  '6_6292'],
 'sc_genes': ['YGR240C', 'YMR205C'],
 'sc_names': ['PFK1', 'PFK2'],
 'pathways_id': ["['GLUCFERMEN-PWY', 'GLYCOLYSIS']",
  "['GLUCFERMEN-PWY', 'GLYCOLYSIS']"],
 'pathways_name': ["['glycolysis I (from glucose 6-phosphate)', 'superpathway of glucose fermentation']",
  "['glycolysis I (from gluc

#How do we choose the structural alignment??
## Oliver:  scer genes have 'ref'

I see a possible bug. Right now in the file 'OG1004_REF_Scer_AF-P15938-F1-model_v2.FASTA'

All the S. cer genes have REF in front of them - not just Scer_AF-P15938-F1-model_v2

REF_Scer_AF-P15938-F1-model_v2
REF_Scer_AF-P20095-F1-model_v2
REF_Scer_AF-P24384-F1-model_v2
REF_Scer_AF-P40459-F1-model_v2
REF_Scer_AF-P53131-F1-model_v2

shouldn't it be:

REF_Scer_AF-P15938-F1-model_v2
Scer_AF-P20095-F1-model_v2
Scer_AF-P24384-F1-model_v2
Scer_AF-P40459-F1-model_v2
Scer_AF-P53131-F1-model_v2

Oliver says that the word 'REF' means it is the Ref species not the ref structure.  The ref structure is just identified in the filename. 


## Jacob - do we need the identifier_proteome_id (if so use record.description.split(' ')[0])

#Doesn't Wrap

#S Pombe not included in sequence alignments

In [37]:
#sc_gene_id_ref_lookup = pickle.load(open(base_dir + os.sep + os.path.normpath('msas/structural/Genes_identifier_dict.pkl'),"rb"))
#sc_ref_gene_id_lookup = dict(zip(sc_gene_id_ref_lookup.values(), sc_gene_id_ref_lookup.keys()))




In [4]:
path_file = "/Users/Oliver/OneDrive - Charité - Universitätsmedizin Berlin/Alphafold/Alphafold_ranked_0_PDB/Output/data"
data = pd.read_csv("../Data/Scer_protein_names.tsv",sep="\t", index_col="OLN")



# Conversion dictionaries
# protein2og = pickle.load(open(os.path.join(path_file,"Genes_scer_dict.pkl"),"rb"))
# protein2file = pickle.load(open(os.path.join(path_file,"files_scer.pkl"),"rb"))
# protein2gene = {data.loc[index,"Swiss-Prot"]:index for index in data.index}

# gene2identifier = {protein2gene[key]:protein2og[key]+"_REF_Scer_"+protein2file[key][:-4] for key in protein2file.keys()}


#I want to go back from Oliver's protein name to the name from the sequence alignment.  

#This is especially hard for S. cer, S.pom and C. alb which have special names from uniprot.  

identifier2gene = 


FileNotFoundError: [Errno 2] No such file or directory: '../Data/Scer_protein_names.tsv'